# Analyzing Travel Times and Distances With Google Maps

This gathers travel times and distances using the [Google Maps Distance Matrix API](https://developers.google.com/maps/documentation/distance-matrix).

Given a list of $O$ origins and $D$ destinations, it calculates a $O$ by $D$ matrix of the respective travel time and distance for each origin and distance. The service [limits our queries](https://developers.google.com/maps/documentation/distance-matrix/usage-and-billing#other-usage-limits) to $O + D \leq 25$ (up to 25 total inputs) and $O \times D \leq 100$ (up to 100 outputs). (We're also arbitrarily rate limited, hence the `sleep` timer.)

Ensure that Python 3.x and the following dependencies are installed. I'm sorry I did not provide more detailed installation and configuration information.

In [ ]:
from config import MAPS_API_KEY as API_KEY
import requests
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
from time import sleep

import plotly.graph_objects as go
import plotly.express as px

Below is an example input URL: multiple origins, one destination.
Here, both origins and destinations are in latitude and longitude coordinates. (The API can take other forms, but we're only using coordinates.) Input format for both of these is `latitude_coord`%2C`longitude_coord`%7C`...`.

In [ ]:
ex_query_url = """https://maps.googleapis.com/maps/api/distancematrix/json
?destinations=40.659569%2C-73.933783%7C40.729029%2C-73.851524%7C40.6860072%2C-73.6334271%7C40.598566%2C-73.7527626
&origins=40.6655101%2C-73.89188969999998
&key=YOUR_API_KEY"""

### Example Query

In [ ]:
formatted_dests = "40.659569%2C-73.933783%7C40.729029%2C-73.851524%7C40.6860072%2C-73.6334271%7C40.598566%2C-73.7527626"
formatted_origs = "40.6655101%2C-73.89188969999998"
example_url = f"https://maps.googleapis.com/maps/api/distancematrix/json?destinations={formatted_dests}&origins={formatted_origs}&key={API_KEY}"

ex_query_url = f"https://maps.googleapis.com/maps/api/distancematrix/json?destinations=40.659569%2C-73.933783%7C40.729029%2C-73.851524%7C40.6860072%2C-73.6334271%7C40.598566%2C-73.7527626&origins=40.6655101%2C-73.89188969999998&key={API_KEY}"
ex_query = requests.request("GET", ex_query_url)
print(ex_query.text)

### Formatting Locations Into API-Specified String
Accepts a DataFrame input (see "Reading and Filtering Input Data") and formats its latitude and loingitude coordinates to prepare it for querying.

In [ ]:
def format_locations(locs: pd.DataFrame) -> str:
    return "".join(
        [
            f"{coords['Latitude']}%2C{coords['Longitude']}%7C"
            for _, coords in locs.iterrows()
        ]
    )

## Homes and Corrals vs. Watering Points

### Reading and Filtering Input Data
Most (if not all) of customizing the query is here, as we get to specify which destinations and origins to calculate distances against.

In [ ]:
df = pd.read_excel("GPS_Points_Master_Sheet.xlsx")
locs = df[["Name", "Type", "Longitude", "Latitude"]]

# Filter to water points (5 total)
# Requires bitwise OR `|` and paranthesis around each conditional, i.e. `(locs['Column name'] == 'Row value") | ...`
dests = locs[
    (locs["Name"] == "Cameron Watering Point")
    | (locs["Name"] == "Gap Watering Point")
    | (locs["Name"] == "Bitter Springs Watering Point")
    | (locs["Name"] == "Marble Canyon Watering Point")
    | (locs["Name"] == "Hidden Springs Watering Point")
]

# Filter to homes and corrals (341 total)
origs = locs.loc[(locs["Name"] == "Home") | (locs["Name"] == "Corral")]

### Querying

In [ ]:
def fetch_distances(formatted_dests: str, formatted_origs: str) -> str:
    return requests.request(
        "GET",
        f"https://maps.googleapis.com/maps/api/distancematrix/json?destinations={formatted_dests}&origins={formatted_origs}&key={API_KEY}",
    )

In [ ]:
MAX_QUERIES = 25 - len(dests)
raw_results = []  # Each query is a Response object

formatted_dests = format_locations(dests)

for i in range(0, len(origs), MAX_QUERIES):
    # Query up to MAX_QUERIES origins starting from the i-th row
    formatted_origs = format_locations(origs.iloc[i : min(i + MAX_QUERIES, len(locs))])
    raw_results.append(fetch_distances(formatted_dests, formatted_origs))
    sleep(0.000001)  # Arbitrary sleep timer to workaround rate limiting

## Processing the Results

The individual raw results are hard to analyze by themselves, so we merge them into the `dests` and `origs` DataFrames. (Remember that we probably made multiple queries to workaround the API limitations.) We also export the raw results in a list as a JSON file and the formatted DataFrames as a CSV/XSLX file.

Most of the formatting is on the locations (destinations and origins), where we merge the API-fetched address with the input location coordinates/point, name, and type. E.g.
```((-111.4136128, 35.8744329), "Cameron Watering Point", "465 US-89, Cameron, AZ 86020, USA")``` (This is not necessarily accurate.)

Note that we take advantage that the queries are in the same order as the spreadsheet, from top to bottom, so we can process the results and input data sequentially without mixing up information/rows.

### Example Formatted Result

In [ ]:
ex_formatted_info = {
    "destinations": [
        # ((<Longitude>, <Latitude>), "Location name", "<Address from query>"),
        # ...
    ],
    "origins": [
        #  ... Same format as that of destinations
    ],
    "info": [
        [
            {  # Travel information from first origin to first destination
                "distance": "4.6 km",
                "duration": "18 mins",
                "status": "OK",
            },
            # ... (Information for following destinations)
        ]
        # ... (Information for following origins)
    ],
}

### Exporting Raw Results

In [ ]:
with open("raw_info.json", "w") as outfile:
    outfile.write("[" + ",".join([res.text for res in raw_results]) + "]")

### Formatting Results

In [ ]:
# To format into # of mins. e.g. input times "23 mins" or "1 hour 3 mins"
def format_time(time: str) -> int:
    min_index = time.index("min")
    if "hour" in time:
        hr_index = time.index("hour")
        time = int(time[:hr_index]) * 60 + int(time[hr_index + len("hour") : min_index])
    else:
        time = int(time[:min_index])

    return time

In [ ]:
# 341 (origins) X 5 (destinations)
time_matrix = np.zeros((len(origs), len(dests)), int)

orig_i = 0
for res_obj in raw_results:
    res = json.loads(res_obj.text)
    for i, dest_infos in enumerate(res["rows"]):
        for dest_i, travel_info in enumerate(res["rows"][i]["elements"]):
            time_matrix[orig_i, dest_i] = format_time(travel_info["duration"]["text"])
        orig_i += 1

for i, col in enumerate(time_matrix.T):
    origs.assign(**{dests["Name"].iloc[i] + " Travel Time (min.)": col})

origs.to_excel("formatted_data.xlsx", index=False)

### Computing Analyses

In [ ]:
median_times = np.median(time_matrix, axis=1)
mean_times = np.mean(time_matrix, axis=1)

origs = origs.assign(**{"Median": median_times.tolist()})
origs = origs.assign(**{"Mean": mean_times.tolist()})

origs.to_excel("formatted_data.xlsx", index=False)

### Analyses Figures
IDK how to neatly plot in the same figure, so two cells each for mean and median.

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot()
n, bins, patches = ax1.hist(
    median_times, bins=20, weights=np.ones(len(mean_times)) / len(mean_times)
)

plt.title("Homes and Corrals vs. Median Travel Time")
plt.xlabel("Median Travel Time to Watering Points (mins.)")
plt.ylabel("% of All Homes/Corrals")

In [ ]:
fig = plt.figure()
ax2 = fig.add_subplot()
n, bins, patches = ax2.hist(
    mean_times, bins=20, weights=np.ones(len(mean_times)) / len(mean_times)
)

plt.title("Homes and Corrals vs. Mean Travel Time")
plt.xlabel("Mean Travel Time to Watering Points (mins.)")
plt.ylabel("% of All Homes/Corrals")

In [ ]:
print(
    f"""
      | Other Information |
      Number of origins: {len(origs)}
      Number of destinations: {len(dests)}
      Standard deviation of median times: {np.std(median_times)}
      Standard deviation of mean times: {np.std(mean_times)}
      """
)

## Geoplotting

In [ ]:
boundary_points = None
with open("boundary.txt") as f:
    point_strs = f.readlines()[0].split(", ")
    boundary_points = np.empty((len(point_strs), 2), dtype=tuple)
    for i, point_str in enumerate(point_strs):
        p = point_str.split(" ")
        boundary_points[i] = np.array([float(p[0]), float(p[1])])

In [ ]:
# Hover labels for plot points
orig_labels = (
    origs["Name"]
    + ", "
    + origs["Type"]
    + ". Min: "
    + time_matrix.min(axis=1).astype(str)
)
dest_labels = dests["Name"] + ", " + dests["Type"]

# Origins (Homes and Corrals)
fig = go.Figure(
    data=go.Scattergeo(
        # locationmode='USA-states',
        lat=origs["Latitude"],
        lon=origs["Longitude"],
        text=orig_labels,
        marker=dict(
            line=dict(width=1, color="rgba(102, 102, 102)"),
            # colorscale='Blues',
            cmin=0,
            color=time_matrix.min(axis=1),
            cmax=time_matrix.min(axis=1).max(),
            colorbar_title="Minimum Time (min.)",
        ),
    )
)

# Destinations (Watering Points)
fig.add_trace(
    go.Scattergeo(
        lat=dests["Latitude"],
        lon=dests["Longitude"],
        text=dest_labels,
        marker=dict(symbol="square", line=dict(width=1, color="rgba(102, 102, 102)")),
    )
)

# Chapter Border
fig.add_trace(
    go.Scattergeo(lat=boundary_points[:, 1], lon=boundary_points[:, 0], mode="lines")
)
fig.update_geos(fitbounds="locations")
fig.update_layout(legend_orientation="h", mapbox_style="open-street-map")
fig.show()

In [ ]:
fig = px.scatter_mapbox(
    origs,
    lat="Latitude",
    lon="Longitude",
    hover_name="Name",
    hover_data=["Type"],
    color_discrete_sequence=["fuchsia"],
)
fig.add_trace(
    go.Scattergeo(lat=boundary_points[:, 1], lon=boundary_points[:, 0], mode="lines")
)
fig.update_layout(
    margin={"r": 0, "t": 0, "l": 0, "b": 0}, mapbox_style="open-street-map"
)
fig.show()

In [ ]:
print(orig_labels)

In [ ]:
fig = go.Figure(
    data=go.Contour(
        z=time_matrix.min(axis=1),
        x=origs["Latitude"],
        y=origs["Longitude"],
        colorscale="Hot",
        contours_coloring="heatmap",
    )
)

fig.show()

## Countour Plot

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Carpet(
        a=[0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3],
        b=[4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6],
        x=[2, 3, 4, 5, 2.2, 3.1, 4.1, 5.1, 1.5, 2.5, 3.5, 4.5],
        y=[1, 1.4, 1.6, 1.75, 2, 2.5, 2.7, 2.75, 3, 3.5, 3.7, 3.75],
        aaxis=dict(tickprefix="a = ", smoothing=0, minorgridcount=9, type="linear"),
        baxis=dict(tickprefix="b = ", smoothing=0, minorgridcount=9, type="linear"),
    )
)
fig.update_layout()
fig.show()